# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [40]:
!cat publications.csv

citation,title,pub_date,venue,paper_url,excerpt
"Tomaz Logar, Joseph Bullock, Edoardo Nemni, Lars Bromley, John A Quinn, Miguel A Luengo-Oroz",PulseSatellite: A Tool Using Human-AI Feedback Loops for Satellite Image Analysis in Humanitarian Contexts,2020,AAAI,https://arxiv.org/pdf/2001.10685.pdf,(PulseSatellite project page)[https://www.unglobalpulse.org/microsite/pulsesatellite/]
"Godliver Owomugisha, Ephraim Nuwamanya, John A Quinn, Michael Biehl, Ernest Mwebaze",Early detection of plant diseases using spectral data,2020,International Conference on Applications of Intelligent Systems,Owomugisha_2020_Spectral.pdf,
"Godliver Owomugisha, Pius KB Mugagga, Friedrich Melchert, Ernest Mwebaze, John A Quinn, Michael Biehl",A low-cost 3-D printed smartphone add-on spectrometer for diagnosis of crop diseases in field,2020,Conference on Computing and Sustainable Societies (COMPASS),Owomugisha_2020_COMPASS.pdf,
"Jeremy Francis Tusubira, Solomon Nsumba, Flavia Ninsiima, Benjamin Akera, Guy Acella

## Import pandas

We are using the very handy pandas library for dataframes.

In [42]:
import pandas as pd

## Import publications table

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [153]:
publications = pd.read_csv("publications.csv", sep=",", header=0)
publications.head()

,citation,title,url_slug,pub_date,venue,paper_url,excerpt
0,"Tomaz Logar, Joseph Bullock, Edoardo Nemni, La...",PulseSatellite: A Tool Using Human-AI Feedback...,NaN,2020,AAAI (Demo Track),https://arxiv.org/pdf/2001.10685.pdf,[PulseSatellite project page](https://www.ungl...
1,"Godliver Owomugisha, Ephraim Nuwamanya, John A...",Early detection of plant diseases using spectr...,NaN,2020,International Conference on Applications of In...,/files/Owomugisha_2020_Spectral.pdf,NaN
2,"Godliver Owomugisha, Pius KB Mugagga, Friedric...",A low-cost 3-D printed smartphone add-on spect...,NaN,2020,Conference on Computing and Sustainable Societ...,/files/Owomugisha_2020_COMPASS.pdf,NaN
3,"Jeremy Francis Tusubira, Solomon Nsumba, Flavi...",Improving In-Field Cassava Whitefly Pest Surve...,NaN,2020,CVPR 2020 Workshops,/files/Tusubira_2020_WhiteflyDetection.pdf,NaN
4,"Pedro J Zufiria, David Pastor-Escuredo, Luis U...",Mobility profiles and calendars for food secur...,NaN,2019,arXiv:1904.08525,https://arxiv.org/pdf/1904.08525,NaN


## Create URL slug from title and year

In [154]:
import re

url_slugs = []
for i in publications.index:
    title_words = publications.title[i].lower().split()
    title_words = [re.sub('[\W_]+', '', w) for w in title_words]
    if len(title_words) > 5:
        title_words = title_words[:5]
    slug = '-'.join(title_words)
    url_slugs.append(slug)

publications['url_slug'] = url_slugs

## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [155]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [156]:
!rm ../_publications/*.md

In [157]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) + '-01-01'
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt)
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [79]:
!ls ../_publications/

2005-a-comparison-of-graphical-and.md
2005-factorial-switching-kalman-filters-for.md
2007-bayesian-condition-monitoring-in-neonatal.md
2007-how-to-be-lost-principled.md
2007-known-unknowns-novelty-detection-in.md
2008-factorial-switching-linear-dynamical-systems.md
2008-fast-committeebased-structure-learning.md
2008-signal-masking-in-gaussian-channels.md
2009-computational-prediction-of-cholera-outbreaks.md
2009-location-segmentation-inference-and-prediction.md
2009-methodologies-for-continuous-cellular-tower.md
2010-automated-visionbased-diagnosis-of-cassava.md
2010-causal-structure-learning-for-famine.md
2010-increasedspecificity-famine-prediction-using-satellite.md
2010-traffic-flow-monitoring-in-crowded.md
2011-causal-relevance-learning-for-robust.md
2011-divergencebased-classification-in-learning-vector.md
2011-learning-of-causal-relations.md
2011-modeling-and-monitoring-crop-disease.md
2011-physiological-monitoring-with-factorial-switching.md
2012-coupling-spatiotemporal-disease-

In [108]:
!cat ../_publications/2020-pulsesatellite-a-tool-using-humanai.md

---
title: "PulseSatellite: A Tool Using Human-AI Feedback Loops for Satellite Image Analysis in Humanitarian Contexts"
collection: publications
permalink: /publication/2020-pulsesatellite-a-tool-using-humanai
excerpt: '[PulseSatellite project page](https://www.unglobalpulse.org/microsite/pulsesatellite/)'
date: 2020-01-01
venue: 'AAAI'
paperurl: 'https://arxiv.org/pdf/2001.10685.pdf'
citation: 'Tomaz Logar, Joseph Bullock, Edoardo Nemni, Lars Bromley, John A Quinn, Miguel A Luengo-Oroz'
---
[PulseSatellite project page](https://www.unglobalpulse.org/microsite/pulsesatellite/)
[pdf](https://arxiv.org/pdf/2001.10685.pdf)


## Retrieve publications from Google Scholar

Can be helpful for retrieving the details to compile ```publications.tsv```.

In [1]:
SCHOLAR_ID = 'd4_ktUIAAAAJ'

from scholarly import scholarly
author = scholarly.search_author_id(SCHOLAR_ID)
author['filled'] = set()
author = scholarly.fill(author)
unfilled_pubs = author['publications']
pubs = [scholarly.fill(p) for p in unfilled_pubs]

In [5]:
for p in pubs:
    author_list = p['bib']['author']
    author_list = author_list.replace(' and ',', ')
    print(author_list)

John A Quinn, Christopher KI Williams, Neil McIntosh
Anna S Law, Yvonne Freer, Jim Hunter, Robert H Logie, Neil Mcintosh, John Quinn
John A Quinn, Rose Nakasi, Pius KB Mugagga, Patrick Byanyima, William Lubega, Alfred Andama
Ernest Mwebaze, Petra Schneider, F-M Schleif, Jennifer R Aduwo, John A Quinn, Sven Haase, Thomas Villmann, Michael Biehl
Christopher Williams, John Quinn, Neil McIntosh
John A Quinn, Christopher KI Williams
John A Quinn, Masashi Sugiyama
Nathan Eagle, Aaron Clauset, John A Quinn
John A Quinn, Marguerite M Nyhan, Celia Navarro, Davide Coluccia, Lars Bromley, Miguel Luengo-Oroz
Nathan Eagle, John A Quinn, Aaron Clauset
John A Quinn, Alfred Andama, Ian Munabi, Fred N Kiwanuka
Song Liu, John A Quinn, Michael U Gutmann, Taiji Suzuki, Masashi Sugiyama
John Alexander Quinn, Kevin Leyton-Brown, Ernest Mwebaze
Jennifer R Aduwo, Ernest Mwebaze, John A Quinn
John Alexander Quinn, Rose Nakibuule
John Quinn, Vanessa Frias-Martinez, Lakshminarayan Subramanian
Godliver Owomugisha

In [ ]:
for p in pubs:
    b = p['bib']
    venue =  '-'
    if 'journal' in b:
        venue = f"{b.get('journal')} ({b.get('volume')}) {b.get('number')}, {b.get('pages')}"
    print(venue)

In [ ]:
for p in pubs:
    print(p['bib']['title'])
    
for p in pubs:
    print(p['bib']['pub_year'])

In [12]:
urls = []
for p in pubs:
    urls.append(p.get('eprint_url'))
print(urls)

['http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.832.5824&rep=rep1&type=pdf',
 'http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.118.5805&rep=rep1&type=pdf',
 'http://www.jmlr.org/proceedings/papers/v56/Quinn16.pdf',
 'https://www.rug.nl/research/portal/files/2489124/2011NeurocompMwebaze.pdf',
 'https://papers.nips.cc/paper/2005/file/aeefb050911334869a7a5d9e4d0e1689-Paper.pdf',
 'http://www.research.ed.ac.uk/portal/files/21892425/knownunknowns.pdf',
 'http://cit.mak.ac.ug/staff/jquinn/papers/PRLetters_LSAnomalyDetection.pdf',
 'https://www.aaai.org/Papers/Symposia/Spring/2009/SS-09-09/SS09-09-005.pdf',
 'https://royalsocietypublishing.org/doi/pdf/10.1098/rsta.2017.0363',
 'http://realitycommons.media.mit.edu/pdfs/pervasive09.pdf',
 'http://air.ug/~jquinn/papers/AutomatedMalariaDiagnosisChapter.pdf',
 'http://www.ms.k.u-tokyo.ac.jp/2014/CDKLIEP2.pdf',
 'https://cit.mak.ac.ug/staff/jquinn/papers/aaai11cropmonitoring.pdf',
 'http://air.ug/mcrops/papers/dma2010.pdf',
 'ht